In [9]:
import pandas as pd
import sqlite3
import json
import pandas as pd
import json
import re


In [4]:
orders = pd.read_csv("orders.csv")
orders.head()  


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [5]:
with open("users.json") as f:
    users = pd.DataFrame(json.load(f))
users.head()  

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [11]:
# Orders + Users merge
orders_users = pd.merge(orders, users, on='user_id', how='left')

# Merge with Restaurants
final_dataset = pd.merge(orders_users, restaurants, on='restaurant_id', how='left')

# Output
print("Final Merged Dataset:")
print(final_dataset.head())


Final Merged Dataset:
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y   cuisine  rating  
0    Restaurant_450   Mexican     3.2  
1    Restaurant_309    Indian     4.5  
2    Restaurant_107   Mexican     4.0  
3    R

In [12]:
final_dataset.to_csv("final_food_delivery_dataset.csv", index=False)
print("Final dataset saved as final_food_delivery_dataset.csv")


Final dataset saved as final_food_delivery_dataset.csv


In [13]:
revenue_by_membership = final_dataset.groupby('membership')['total_amount'].sum()
print("Revenue by Membership:")
print(revenue_by_membership)


Revenue by Membership:
membership
Gold       3975364.89
Regular    4036259.23
Name: total_amount, dtype: float64


In [14]:
revenue_by_city = final_dataset.groupby('city')['total_amount'].sum()
print("\nRevenue by City:")
print(revenue_by_city)



Revenue by City:
city
Bangalore    2206946.58
Chennai      1990513.03
Hyderabad    1889366.58
Pune         1924797.93
Name: total_amount, dtype: float64


In [15]:
avg_order_by_cuisine = final_dataset.groupby('cuisine')['total_amount'].mean()
print("\nAverage Order Value by Cuisine:")
print(avg_order_by_cuisine)



Average Order Value by Cuisine:
cuisine
Chinese    798.389020
Indian     798.466011
Italian    799.448578
Mexican    808.021344
Name: total_amount, dtype: float64


In [16]:
user_total_orders = final_dataset.groupby('user_id')['total_amount'].sum()
high_value_users = user_total_orders[user_total_orders > 1000]
print("\nNumber of distinct users with orders > 1000:", high_value_users.count())



Number of distinct users with orders > 1000: 2544


In [17]:
# Create rating bins
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0-3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"]
final_dataset['rating_range'] = pd.cut(final_dataset['rating'], bins=bins, labels=labels)

revenue_by_rating_range = final_dataset.groupby('rating_range')['total_amount'].sum()
print("\nRevenue by Restaurant Rating Range:")
print(revenue_by_rating_range)



Revenue by Restaurant Rating Range:
rating_range
3.0-3.5    2136772.70
3.6-4.0    1717494.41
4.1-4.5    1960326.26
4.6-5.0    2197030.75
Name: total_amount, dtype: float64


In [18]:
gold_orders = final_dataset[final_dataset['membership'] == 'Gold']
gold_avg_by_city = gold_orders.groupby('city')['total_amount'].mean()
print("\nGold Members – Average Order Value by City:")
print(gold_avg_by_city)



Gold Members – Average Order Value by City:
city
Bangalore    793.223756
Chennai      808.459080
Hyderabad    806.421034
Pune         781.162243
Name: total_amount, dtype: float64


In [19]:
total_orders = len(final_dataset)
gold_orders_count = len(final_dataset[final_dataset['membership'] == 'Gold'])
percent_gold_orders = round((gold_orders_count / total_orders) * 100)
print("\nPercentage of Orders by Gold Members:", percent_gold_orders, "%")



Percentage of Orders by Gold Members: 50 %


In [20]:
restaurant_orders = final_dataset.groupby('restaurant_name_y').agg({'total_amount':'mean','order_id':'count'})
top_restaurants = restaurant_orders[restaurant_orders['order_id'] < 20].sort_values('total_amount', ascending=False)
print("\nTop Revenue Restaurants with less than 20 orders:")
print(top_restaurants.head())



Top Revenue Restaurants with less than 20 orders:
                   total_amount  order_id
restaurant_name_y                        
Restaurant_294      1040.222308        13
Restaurant_262      1029.473333        18
Restaurant_77       1029.180833        12
Restaurant_193      1026.306667        15
Restaurant_7        1002.140625        16
